In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd 
import seaborn as sns


from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import os

In [2]:
train = pd.read_csv('train_prepared.csv', index_col=[0], parse_dates=[0])
#test = pd.read_csv('test_prepared.csv', index_col=[0], parse_dates=[0])

train.head()

,Passengers,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,hour,t1,t2,t3,exp_0_01,exp_0_03,exp_0_08
Month,,,,,,,,,,,,,,,
1949-01-01,112,1949,1,1,1,1,53,5,0,0.0,0.0,0.0,112.000000,112.000000,112.000000
1949-02-01,118,1949,2,1,1,32,5,1,0,112.0,0.0,0.0,112.060000,112.180000,112.480000
1949-03-01,132,1949,3,1,1,60,9,1,0,118.0,112.0,0.0,112.259400,112.774600,114.041600
1949-04-01,129,1949,4,1,2,91,13,4,0,132.0,118.0,112.0,112.426806,113.261362,115.238272
1949-05-01,121,1949,5,1,2,121,17,6,0,129.0,132.0,118.0,112.512538,113.493521,115.699210


Сделаем кроссвалидацию и сохраним оценки

In [3]:
lr = RandomForestRegressor()
#sc = StandardScaler()
X = train.drop('Passengers', 1).values
y = train['Passengers'].values
#scaler = sc.fit(X)
#X_scaled = sc.transform(X)

In [4]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(10)

CV_scores_lr = []
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = RandomForestRegressor(n_estimators=100, max_depth=16)
    lr.fit(X_train, y_train)
    preds = lr.predict(X_test)
    CV_scores_lr.append(r2_score(y_test, preds))

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] TEST: [20 21 22 23 24 25 26 27 28 29 30]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30] TEST: [31 32 33 34 35 36 37 38 39 40 41]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41] TEST: [42 43 44 45 46 47 48 49 50 51 52]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52] TEST: [53 54 55 56 57 58 59 60 61 62 63]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63] TEST: [64 65 66 67 68 69 70 71 72 73 74]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 

In [5]:
CV_scores_lr

[0.33520857777221835,
 0.2362389418655303,
 -0.3910167740336965,
 0.24347077787999594,
 0.2819494689845746,
 -0.6191020591107343,
 -0.07156901304998264,
 0.18005782414066163,
 0.6953419279534947,
 0.6018631605120882]

Сохраним обученную модель

In [6]:
import dill

In [7]:
lr = RandomForestRegressor(n_estimators=100, max_depth=16)
lr.fit(X, y)
with open("models/RF.model", "wb") as f:
    dill.dump(lr, f)

### GB regression

In [8]:
tscv = TimeSeriesSplit(10)

CV_scores_lr_l2 = []
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = GradientBoostingRegressor(max_depth=3, n_estimators=50)
    lr.fit(X_train, y_train)
    preds = lr.predict(X_test)
    CV_scores_lr_l2.append(r2_score(y_test, preds))

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] TEST: [20 21 22 23 24 25 26 27 28 29 30]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30] TEST: [31 32 33 34 35 36 37 38 39 40 41]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41] TEST: [42 43 44 45 46 47 48 49 50 51 52]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52] TEST: [53 54 55 56 57 58 59 60 61 62 63]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63] TEST: [64 65 66 67 68 69 70 71 72 73 74]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 

In [9]:
lr = GradientBoostingRegressor(max_depth=3, n_estimators=50)
lr.fit(X, y)
with open("models/GBM.model", "wb") as f:
    dill.dump(lr, f)

In [10]:
pd.DataFrame({'RF': CV_scores_lr,
              'GBM': CV_scores_lr_l2}).to_csv("./models/CV_Trees.csv")

In [11]:
pd.DataFrame({'RF': CV_scores_lr,
              'GBM': CV_scores_lr_l2})

,RF,GBM
0,0.335209,0.043747
1,0.236239,0.328165
2,-0.391017,-0.006633
3,0.243471,0.303490
4,0.281949,0.319603
5,-0.619102,-0.732758
6,-0.071569,0.108358
7,0.180058,0.253387
8,0.695342,0.777585
9,0.601863,0.697987
